In [1]:
import json
import operator
import sys
import os
from web3 import HTTPProvider
from web3 import Web3
from web3.middleware import geth_poa_middleware

from src.receipt_log_handler import EthReceiptLogHandler

In [2]:
provider_url = "https://bsc-dataseed4.binance.org/"
web3 = Web3(HTTPProvider(provider_url))
web3.middleware_onion.inject(geth_poa_middleware, layer=0)

print(f"Success?: {web3.is_connected()}")

Success?: True


In [3]:
NUM_BLOCKS = 1000

# Uniswap

## 1. Pool info
Lâý thông tin của lp_token từ id 

In [4]:
masterchefv2_addr = "0xa5f8C5Dbd5F286960b9d90548680aE5ebFf07652"
pid = 3
with open('./abi/masterchef_abi.json', 'r') as f:
    abi = json.load(f)
masterchefv2 = web3.eth.contract(abi=abi, address=masterchefv2_addr)

# 1. pool info
pool_info_array = masterchefv2.functions.poolInfo(pid).call()
pool_info = {
    'accCakePerShare': pool_info_array[0],
    'lastRewardBlock': pool_info_array[1],
    'allocPoint': pool_info_array[2],
    'totalBoostedShare': pool_info_array[3],
    'isRegular': pool_info_array[4],
}

# pool explanation. Reference: source code of the smart contract
pool_explanation = {
    'accCakePerShare': "Accumulated CAKEs per share, times 1e12",
    'lastRewardBlock': "Last block number that pool update action is executed",
    'allocPoint': "The amount of allocation points assigned to the pool",
    'totalBoostedShare': "The total amount of user shares in each pool. After considering the share boosts",
    'isRegular': "The flag to set pool is regular or special",
}

result = {
    'poolInfo': pool_info,
    'poolExplanation': pool_explanation,
}
json_obj = json.dumps(result, indent=2)
with open('./uniswap1.json', 'w') as f:
    f.write(json_obj)
result

{'poolInfo': {'accCakePerShare': 5848148801503265166,
  'lastRewardBlock': 36716890,
  'allocPoint': 0,
  'totalBoostedShare': 37481995681978615045559,
  'isRegular': True},
 'poolExplanation': {'accCakePerShare': 'Accumulated CAKEs per share, times 1e12',
  'lastRewardBlock': 'Last block number that pool update action is executed',
  'allocPoint': 'The amount of allocation points assigned to the pool',
  'totalBoostedShare': 'The total amount of user shares in each pool. After considering the share boosts',
  'isRegular': 'The flag to set pool is regular or special'}}

## 2. LP token address
Tìm địa chỉ lp_token.

In [5]:
# 2. lp token address
lp_addr = masterchefv2.functions.lpToken(pid).call()
print(f"LP token address: {lp_addr}")
result = {
    'lpTokenAddress': lp_addr,
}

json_obj = json.dumps(result, indent=2)
with open('./uniswap2.json', 'w') as f:
    f.write(json_obj)

LP token address: 0x58F876857a02D6762E0101bb5C46A8c1ED44Dc16


## 3. Reserves of LP pair
Tìm địa chỉ lp_token. Dựa vào `abi/lp_token_abi.json` và địa chỉ lp_token tìm được lấy địa chỉ cặp swap và thông tin về reserves của cặp swap

In [6]:
# 3. Reserves
with open('./abi/lp_token_abi.json', 'r') as f:
    abi = json.load(f)
lp_contract = web3.eth.contract(abi=abi, address = lp_addr)

token0 = lp_contract.functions.token0().call()
token1 = lp_contract.functions.token1().call()
print(f"token 0: {token0} \ntoken 1: {token1}")

reserves_info = lp_contract.functions.getReserves().call()
reserve0 = reserves_info[0]
reserve1 = reserves_info[1]
print(f"reserve 0: {reserve0}\nreserve 1: {reserve1}")

result = {
    token0: reserve0,
    token1: reserve1,
}
json_obj = json.dumps(result, indent=2)
with open('./uniswap3.json', 'w') as f:
    f.write(json_obj)

token 0: 0xbb4CdB9CBd36B01bD1cBaEBF2De08d9173bc095c 
token 1: 0xe9e7CEA3DedcA5984780Bafc599bD69ADd087D56
reserve 0: 9915460083244449869770
reserve 1: 3927264198498533991813009


## 4. Crawl 5 events data
Crawl dữ liệu event Swap, Approval, Burn, Mint và Transfer của lp_token trong 1000 blocks gần nhất

In [7]:
# 4. Events

# The lp_events.json abi file only contains the 5 required events:
# Swap, Approval, Mint, Burn and Transfer
with open("./lp_events.json", "r") as f:
    event_abi = json.load(f)

handler = EthReceiptLogHandler()

# info about the events based on the ABI
event_abi_info = handler.build_list_info_event(event_abi)
print(f"Event abi info: \n{event_abi_info}")

event_hash = [event_info[1] for event_info in event_abi_info]
print(f"Event hash:\n{event_hash}")

event_subscriber = {info[1]:info[0] for info in event_abi_info}
print(f"Event subsriber:\n{event_subscriber}")

Event abi info: 
[[<model.receipt_log.EventSubscriber object at 0x111bd5270>, '0xd78ad95fa46c994b6551d0da85fc275fe613ce37657fb8d5e3d130840159d822', ['sender', 'to'], 'Swap'], [<model.receipt_log.EventSubscriber object at 0x111bd6a70>, '0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925', ['owner', 'spender'], 'Approval'], [<model.receipt_log.EventSubscriber object at 0x111bd69b0>, '0xdccd412f0b1252819cb1fd330b93224ca42612892bb3f4f789976e6d81936496', ['sender', 'to'], 'Burn'], [<model.receipt_log.EventSubscriber object at 0x111bd5660>, '0x4c209b5fc8ad50758f13e2e1088ba56a560dff690a1c6fef26394f4c03821c4f', ['sender'], 'Mint'], [<model.receipt_log.EventSubscriber object at 0x111bd6470>, '0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef', ['from', 'to'], 'Transfer']]
Event hash:
['0xd78ad95fa46c994b6551d0da85fc275fe613ce37657fb8d5e3d130840159d822', '0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925', '0xdccd412f0b1252819cb1fd330b93224ca4261

In [8]:
end_block = web3.eth.block_number
start_block = end_block - NUM_BLOCKS + 1
blocks_info = {
    'startBlock': start_block,
    'endBlock': end_block,
}

#create filter
filter_params = {
    "fromBlock": start_block,
    "toBlock": end_block,
    "topics": [event_hash],
    "address":[lp_addr]
}

logs_filter = web3.eth.filter(filter_params)  
event_logs = logs_filter.get_all_entries()  # get receipt logs of events
events_list = []
for event_log in event_logs:
    log = handler.web3_dict_to_receipt_log(event_log)
    eth_event = handler.extract_event_from_log(
        log, event_subscriber[log.topics[0]])
    if eth_event is not None:
        eth_event_dict = handler.eth_event_to_dict(eth_event)
        events_list.append(eth_event_dict)

web3.eth.uninstall_filter(logs_filter.filter_id)

print(f"Number of events: {len(events_list)}")

result = {
    'blocksInfo': blocks_info,
    'events': events_list
}

json_obj = json.dumps(result, indent=2)
with open('./uniswap4.json', 'w') as f:
    f.write(json_obj)

events_list[:2]

Number of events: 148


[{'type': 'event',
  'event_type': 'SWAP',
  'contract_address': '0x58f876857a02d6762e0101bb5c46a8c1ed44dc16',
  'transaction_hash': '0xfeb5223f2f047cca0d543559be6e932475117d127d0a1239c310882d2e76ea3c',
  'log_index': 158,
  'block_number': 36721233,
  'sender': '0x10ed43c718714eb63d5aa57b78b54704e256024e',
  'to': '0x10ed43c718714eb63d5aa57b78b54704e256024e',
  'amount0In': '0',
  'amount1In': '6261072748145606401',
  'amount0Out': '15981145022413754',
  'amount1Out': '0'},
 {'type': 'event',
  'event_type': 'SWAP',
  'contract_address': '0x58f876857a02d6762e0101bb5c46a8c1ed44dc16',
  'transaction_hash': '0xb1adad1bc9a7541dd0de1cdff732b744c108ceab58a4680b3eb9e8f763102541',
  'log_index': 171,
  'block_number': 36721265,
  'sender': '0x1111111254eeb25477b68fb85ed929f73a960582',
  'to': '0xa7ca2c8673bcfa5a26d8ceec2887f2cc2b0db22a',
  'amount0In': '29786873896003838',
  'amount1In': '0',
  'amount0Out': '0',
  'amount1Out': '11611571405661355164'}]

## 5. Crawl transaction data from the events above
Crawl dữ liệu transaction tương ứng với dữ liệu events.

In [10]:
tx_hashes_list = [event['transaction_hash'] for event in events_list]
# tx_list = [web3.eth.get_transaction(tx_hash) for tx_hash in tx_hashes_list]
tx_list = list()
for i, tx_hash in enumerate(tx_hashes_list):
    tx = json.loads(web3.to_json(web3.eth.get_transaction(tx_hash)))
    tx_list.append(tx)
    # print to console each 50th transactions to track the progress
    if i%50 == 0:
        print(f"Transaction {i} done")
    
result = {
    'blocksInfo': blocks_info,
    'transactions': tx_list
}
print(f"Number of transactions: {len(tx_list)}")

Transaction 0 done
Transaction 50 done
Transaction 100 done
Number of transactions: 148


In [11]:
result = {
    'blocksInfo': blocks_info,
    'transactions': tx_list
}
json_obj = json.dumps(result, indent=2)
with open('./uniswap5.json', 'w') as f:
    f.write(json_obj)

## 6. User with the most transactions from
Tìm user giao dịch nhiều nhất

In [13]:
# 6. User with the most transactions from
from_user_tx = dict()
for tx in tx_list:
    from_user = tx['from']
    if from_user not in from_user_tx:
        from_user_tx[from_user] = 1
    else:
        from_user_tx[from_user] += 1

max_from_user, max_frequency = max(from_user_tx.items(), key=operator.itemgetter(1))

result = dict()
result['blocksInfo'] = blocks_info
result.update({
    'maxFrequency': max_frequency,
    'wallet': max_from_user
})
json_obj = json.dumps(result, indent=2)
with open('./uniswap6.json', 'w') as f:
    f.write(json_obj)

## 7. Info of the user above
Tìm thông tin của user ở câu 6 dựa trên địa chỉ masterchef và abi `abi/masterchef_abi.json`

In [14]:
# 7. User info
user_info = masterchefv2.functions.userInfo(pid, max_from_user).call()
user_info

result = {
    'userInfo': user_info,
    'infoExplanation': {
        'amount': "amount of LP token that the user provide",
        'amount': "to calculate correct amount for reward pending"
    }
}
json_obj = json.dumps(result, indent=2)
with open('./uniswap7.json', 'w') as f:
    f.write(json_obj)